In [100]:
import pandas as pd
import numpy as np

## Importing the Data obtained from Compustat

In [149]:
df = pd.read_csv("FF_data.csv")

In [150]:
# Finding and extracting all unique industry codes
sics = df['sic'].unique()
np.savetxt('sic.txt',sics,newline='\n', fmt='%d')

## Importing the Industry Data and Creating Industry Variables

In [151]:
# Importing data for the industry variables
df_ind = pd.read_csv("FF_industry_data.csv")
sics_i = df_ind['sic'].unique()
# Checking for missing sics
print(len(sics),len(sics_i))
mv = np.setdiff1d(sics,sics_i)
print(mv)
print(len(df))

376 374
[100 700]
33255


In [152]:
# Adding the industry total to the original data
grouped = df_ind.groupby(['sic', 'fyear']).agg({'emp': 'sum', 'sale': 'sum', 'xopr': 'sum', 'ni': 'sum',
                                               'at': 'sum', 'ceq': 'sum', 'invt': 'sum', 'ap': 'sum',
                                               'lt': 'sum', 'xint': 'sum'})
grouped = grouped.rename(columns={'emp': 'sum_emp', 'sale': 'sum_sale', 'xopr': 'sum_xopr', 'ni': 'sum_ni',
                                 'at': 'sum_at', 'ceq': 'sum_ceq', 'invt': 'sum_invt', 'ap': 'sum_ap',
                                 'lt': 'sum_lt', 'xint': 'sum_xint'})
df = df.merge(grouped, on=['sic','fyear'])

## Creation of Lagged Variables

In [153]:
# Sort dataframe by cik and year
df = df.sort_values(by=['cik','fyear'])

# Specifying number of lags
one_lag = 1
two_lag = 2
three_lag = 3
four_lag = 4

# Creating lagged columns to facilitate later calculations starting with one year lag
for var in ['prcc_f', 'oancf', 'ni', 'emp', 'ib', 'csho', 'cogs', 'ob', 'mrc1', 'mrc2', 'mrc3', 'mrc4', 'mrc5',
            'txp', 'xopr', 'ceq', 'ap', 'xint', 'sum_at', 'sum_xopr', 'sum_ni', 'sum_ceq', 'sum_invt',
            'sum_ap', 'sum_lt', 'sum_xint', 'sum_sale', 'sum_emp']:
    for lag in range(1, one_lag+1):
        df[f'{var}_lag{lag}'] = df.groupby('cik')[var].shift(lag)
        
# Two year lag
for var in ['rect', 'invt']:
    for lag in range(1, two_lag+1):
        df[f'{var}_lag{lag}'] = df.groupby('cik')[var].shift(lag)
        
# Three year lag
for var in ['capx', 'sale', 'au']:
    for lag in range(1, three_lag+1):
        df[f'{var}_lag{lag}'] = df.groupby('cik')[var].shift(lag)
        
# Four year lag
for var in ['act', 'che', 'lct', 'dlc', 'at', 'ivao', 'lt', 'dltt', 'ivst', 'pstk']:
    for lag in range(1, four_lag+1):
        df[f'{var}_lag{lag}'] = df.groupby('cik')[var].shift(lag)

In [154]:
df.shape

(33179, 151)

## Feature Engineering

In [155]:
# Creating dataframe for the financial fundamentals
FF = pd.DataFrame()
# CIK
FF['cik'] = df['cik']
# Year
FF['year'] = df['fyear']
# SIC
FF['SIC'] = df['sic']

### Perols 2011

In [156]:
# AR to Sales (ARtS)
FF["ARtS"] = df['rect'] / df['sale']
# AR to Total Assets (ARtTA)
FF["ARtTA"] = df['rect'] / df['at']
# Allowance of doubtfoul Accounts (AFDA)
FF['AFDA'] = df['recd']
# AFDA to AR
FF['AFDAtAR'] = df['recd'] / df['rect']
# AFDA to Net Sales 
FF['AFDAtNS'] = df['recd'] / df['sale']
# Altman Z-score
FF['Zscore'] = (3.3 * (df['ib'] + df['xint'] + df['txt']) / df['at'] + 0.999 * (df['sale'] / df['at']) +
                0.6 * df['csho'] * (df['prcc_f'] / df['lt']) + 1.2 * (df['wcap']/df['at']) + 1.4 * (df['re']/df['at']))
# Big 4 auditor
FF['Big4'] = df['au'].isin([1, 2, 3, 4, 5, 6, 7, 8]).astype(int)
# Current minus Prior Year Inventory to Sales
FF['cItS'] = df['invt']/df['sale'] - df['invt_lag1']/df['sale_lag1']
# Days in Receivables Index
FF['DiRI'] = df['rect']/df['sale'] - df['rect_lag1']/df['sale_lag1']
# Debt to Equity
FF['DtE'] = df['lt']/df['ceq']
# Demand for Financing (ex ante)
FF['DfF'] = np.where((df['oancf']-(df['capx_lag3']+df['capx_lag2']+df['capx_lag1'])/3)/df['act'] < -0.5, 1, 0)
# Declining Cash Sales Dummy
FF['DCS'] = np.where((df['sale']-(df['rect']-df['rect_lag1'])) < (df['sale_lag1']-(df['rect_lag1']-df['rect_lag2'])), 1, 0) 
# Fixed Assets to Total Assets
FF['FAtTT'] =  df['ppegt'] / df['at']
# Four-Year Geometric Sales Growth Rate
FF['4GSGR'] = (((df['sale']/df['sale_lag3'])**(1/4))-1)
# Gross Margin
FF['GM'] = (df['sale']-df['cogs']) / df['sale']
# Holding Period Return in the Violation Period
FF['HPRiVP'] = (df['prcc_f']-df['prcc_f_lag1']) / df['prcc_f']
# Industry ROE minus Firm ROE
FF['cROE'] = (df['sum_ni'] / df['sum_ceq']) - (df['ni'] / df['ceq'])
# Inventory to Sales
FF['ItS'] = df['invt'] / df['sale']
# Positive Accruals dummy
FF['PA'] = np.where((df['ib']-df['oancf'] > 0) & (df['ib_lag1']-df['oancf_lag1'] > 0), 1, 0)
# Prior Year ROA to Total Assets Current Year
FF['ROAtTA'] = (df['ni_lag1'] / df['at_lag1']) / df['at']
# Property Plant and Equipment to Total Assets
FF['PPEtTA'] = df['ppent'] / df['at']
# Sales to Total Assets
FF['StTA'] = df['sale'] / df['at']
# The Number of Auditor Turnovers
FF['ATO'] = np.where(df['au'] != df['au_lag1'], 1, 0) + \
            np.where(df['au_lag1'] != df['au_lag2'], 1, 0) + \
            np.where(df['au_lag2'] != df['au_lag3'], 1, 0)
# Times Interest Earned
FF['TIE'] = (df['ib'] + df['xint'] + df['txt']) / df['xint']
# Total Accruals to Total Assets
FF['TACtTA'] = (df['ib'] - df['oancf']) / df['at']
# Total Debt to total Assets
FF['TDtTA'] = df['lt'] / df['at']
# Total Discretionary Accrual
FF['TDA'] = (((((df['act_lag1'] - df['che_lag1']) - (df['lct_lag1'] - df['dlc_lag1'])) - 
                ((df['act_lag2'] - df['che_lag2']) - (df['lct_lag2'] - df['dlc_lag2']))) +
              (((df['at_lag1'] - df['act_lag1'] - df['ivao_lag1']) - (df['lt_lag1'] - df['lct_lag1'] - df['dltt_lag1'])) - 
               ((df['at_lag2'] - df['act_lag2'] - df['ivao_lag2']) - (df['lt_lag2'] - df['lct_lag2'] - df['dltt_lag2']))) +
              (((df['ivst_lag1'] + df['ivao_lag1']) - (df['dltt_lag1'] + df['dlc_lag1'] + df['pstk_lag1'])) - 
               ((df['ivst_lag2'] + df['ivao_lag2']) - (df['dltt_lag2'] + df['dlc_lag2'] + df['pstk_lag2'])))) +
            ((((df['act_lag2'] - df['che_lag2']) - (df['lct_lag2'] - df['dlc_lag2'])) - 
                ((df['act_lag3'] - df['che_lag3']) - (df['lct_lag3'] - df['dlc_lag3']))) +
              (((df['at_lag2'] - df['act_lag2'] - df['ivao_lag2']) - (df['lt_lag2'] - df['lct_lag2'] - df['dltt_lag2'])) - 
               ((df['at_lag3'] - df['act_lag3'] - df['ivao_lag3']) - (df['lt_lag3'] - df['lct_lag3'] - df['dltt_lag3']))) +
              (((df['ivst_lag2'] + df['ivao_lag2']) - (df['dltt_lag2'] + df['dlc_lag2'] + df['pstk_lag2'])) - 
               ((df['ivst_lag3'] + df['ivao_lag3']) - (df['dltt_lag3'] + df['dlc_lag3'] + df['pstk_lag3'])))) +
            ((((df['act_lag3'] - df['che_lag3']) - (df['lct_lag3'] - df['dlc_lag3'])) - 
                ((df['act_lag4'] - df['che_lag4']) - (df['lct_lag4'] - df['dlc_lag4']))) +
              (((df['at_lag3'] - df['act_lag3'] - df['ivao_lag3']) - (df['lt_lag3'] - df['lct_lag3'] - df['dltt_lag3'])) - 
               ((df['at_lag4'] - df['act_lag4'] - df['ivao_lag4']) - (df['lt_lag4'] - df['lct_lag4'] - df['dltt_lag4']))) +
              (((df['ivst_lag3'] + df['ivao_lag3']) - (df['dltt_lag3'] + df['dlc_lag3'] + df['pstk_lag3'])) - 
               ((df['ivst_lag4'] + df['ivao_lag4']) - (df['dltt_lag4'] + df['dlc_lag4'] + df['pstk_lag4'])))))
# Unexpected Employee Productivity
FF['UEP'] = (((df['sale']/df['emp'] - df['sale_lag1']/df['emp_lag1'])/df['sale_lag1']/df['emp_lag1']) -
            ((df['sum_sale']/df['sum_emp'] - df['sum_sale_lag1']/df['sum_emp_lag1'])/df['sum_sale_lag1']/df['sum_emp_lag1']))
# Value of Issued Securities to Market Value
FF['IStMV'] = np.where(df['cshi'] > 0, df['cshi'] * df['prcc_f'] / (df['csho'] * df['prcc_f']), 
                np.where(df['csho'] - df['csho_lag1'] > 0, ((df['csho'] - df['csho_lag1']) * df['prcc_f']) / 
                (df['csho'] * df['prcc_f']), 0))
# Whether Accounts Receivable > 1.1 of Last Year's
FF['IoAR'] = np.where((df['rect'] - df['rect_lag1']) > 1.1, 1,0)
# Whether Gross Margin Percent > 1.1 of Last Year's
FF['IoGMP'] = np.where(((df['sale'] - df['cogs']) / df['sale']) / ((df['sale_lag1']- df['cogs_lag1']) / df['sale_lag1']) 
                       > 1.1, 1, 0)
# Whether New Securities were Issued
FF['NewS'] = np.where((df['csho'] - df['csho_lag1'] > 0) | (df['cshi'] > 0), 1, 0)
# Whether SIC Code Larger (Smaller) than 2999 (4000)
FF['dSIC'] = np.where((df['sic'] > 2999) & (df['sic'] < 4000),1,0)
# Whether firm was listed on AMEX
FF['AMEX'] = np.where((df['exchg'] == 5) | (df['exchg'] == 15) | (df['exchg'] == 16) |
                      (df['exchg'] == 17) | (df['exchg'] == 18), 1, 0)

### Dechow et al. 2011

In [157]:
# Abnormal Change in Order Backlog
FF['ACiOB'] = ((df['ob']-df['ob_lag1']/df['ob_lag1']) - ((df['sale']-df['sale_lag1'])/df['sale_lag1']))
# Actual Issuance
FF['AI'] = np.where((df['sstk'] > 0) | (df['dltis'] > 0), 1, 0)
# Book-to-Market
FF['BtM'] = df['ceq'] / (df['csho'] * df['prcc_f'])
# RSST Accruals
FF['RSST'] = (((((df['act'] - df['che']) - (df['lct'] - df['dlc'])) - 
                ((df['act_lag1'] - df['che_lag1']) - (df['lct_lag1'] - df['dlc_lag1']))) +
              (((df['at'] - df['act'] - df['ivao']) - (df['lt'] - df['lct'] - df['dltt'])) - 
               ((df['at_lag1'] - df['act_lag1'] - df['ivao_lag1']) - (df['lt_lag1'] - df['lct_lag1'] - df['dltt_lag1']))) +
              (((df['ivst'] + df['ivao']) - (df['dltt'] + df['dlc'] + df['pstk'])) - 
               ((df['ivst_lag1'] + df['ivao_lag1']) - (df['dltt_lag1'] + df['dlc_lag1'] + df['pstk_lag1'])))) / 
              ((df['at']+df['at_lag1'])/2))
# Change in Free Cash Flows
FF['CiFCF'] = (((df['ib'] - ((((df['act'] - df['che']) - (df['lct'] - df['dlc'])) - 
                ((df['act_lag1'] - df['che_lag1']) - (df['lct_lag1'] - df['dlc_lag1']))) +
              (((df['at'] - df['act'] - df['ivao']) - (df['lt'] - df['lct'] - df['dltt'])) - 
               ((df['at_lag1'] - df['act_lag1'] - df['ivao_lag1']) - (df['lt_lag1'] - df['lct_lag1'] - df['dltt_lag1']))) +
              (((df['ivst'] + df['ivao']) - (df['dltt'] + df['dlc'] + df['pstk'])) - 
               ((df['ivst_lag1'] + df['ivao_lag1']) - (df['dltt_lag1'] + df['dlc_lag1'] + df['pstk_lag1']))))) / 
               ((df['at'] + df['at_lag1'])/2)) -
              ((df['ib_lag1'] - ((((df['act_lag1'] - df['che_lag1']) - (df['lct_lag1'] - df['dlc_lag1'])) - 
                ((df['act_lag2'] - df['che_lag2']) - (df['lct_lag2'] - df['dlc_lag2']))) +
              (((df['at_lag1'] - df['act_lag1'] - df['ivao_lag1']) - (df['lt_lag1'] - df['lct_lag1'] - df['dltt_lag1'])) - 
               ((df['at_lag2'] - df['act_lag2'] - df['ivao_lag2']) - (df['lt_lag2'] - df['lct_lag2'] - df['dltt_lag2']))) +
              (((df['ivst_lag1'] + df['ivao_lag1']) - (df['dltt_lag1'] + df['dlc_lag1'] + df['pstk_lag1'])) - 
               ((df['ivst_lag2'] + df['ivao_lag2']) - (df['dltt_lag2'] + df['dlc_lag2'] + df['pstk_lag2']))))) /
              ((df['at_lag1']+df['at_lag2'])/2)))
# Change in Inventory
FF['CiI'] = (df['invt'] - df['invt_lag1'] / ((df['at']+df['at_lag1'])/2))
# Change in Operating Lease Activity
FF['CiOLA'] = ((df['mrc1']/1.1 + df['mrc2']/(1.1**2) + df['mrc3']/(1.1**3) + df['mrc4']/(1.1**4) + df['mrc5']/(1.1**5)) -
               (df['mrc1_lag1']/1.1 + df['mrc2_lag1']/(1.1**2) + df['mrc3_lag1']/(1.1**3) + 
                df['mrc4_lag1']/(1.1**4) + df['mrc5_lag1']/(1.1**5))) / ((df['at']+df['at_lag1'])/2)
# Change in Receivables
FF['CiR'] = (df['rect'] - df['rect_lag1']) / ((df['at']+df['at_lag1'])/2)
# Deferred Tax Expense
FF['DTaxE'] = df['txdi'] / df['at_lag1']
# Earnings to Price
FF['EtP'] = df['ib'] / (df['csho'] * df['prcc_f'])
# Existence of Operating Leases
FF['EoOL'] = np.where((df['mrc1'] > 0) | (df['mrc2'] > 0) | (df['mrc3'] > 0) | 
                      (df['mrc4'] > 0) | (df['mrc5'] > 0), 1, 0)
# Level of Finance Raised
FF['LoFR'] = df['fincf'] / ((df['at']+df['at_lag1'])/2)
# Leverage
FF['Lev'] = df['dltt'] / df['at']
# Percentage of Change in Cash Margin
FF['PCiCM'] = (((1 - (df['cogs'] + (df['invt'] - df['invt_lag1'])) / (df['sale'] - (df['rect'] - df['rect_lag1']))) -
                (1 - (df['cogs_lag1'] + (df['invt_lag1'] - df['invt_lag2'])) / (df['sale_lag1'] - (df['rect_lag1'] - df['rect_lag2'])))) / (1 - (df['cogs_lag1'] + (df['invt_lag1'] - df['invt_lag2'])) / (df['sale_lag1'] - (df['rect_lag1'] - df['rect_lag2'])))) 
# Percentage Change in Cash Sales
FF['PCiCS'] = (((df['sale'] - (df['rect'] - df['rect_lag1'])) - (df['sale_lag1'] - (df['rect_lag1'] - df['rect_lag2']))) / (df['sale_lag1'] - (df['rect_lag1'] - df['rect_lag2'])))
# Soft Assets
FF['SA'] = (df['at'] - df['ppent'] - df['che']) / ((df['at']+df['at_lag1'])/2)
# Working Capital Accruals
FF['WCA'] = ((((df['act'] - df['act_lag1']) - (df['che'] - df['che_lag1'])) - ((df['lct'] - df['lct_lag1']) - (df['dlc'] - df['dlc_lag1']) - (df['txp'] - df['txp_lag1'])) - df['dp']) / ((df['at']+df['at_lag1'])/2))

### Cecchini et al. 2010

In [158]:
# Sales
FF['Sales'] =  df['sale']
# Change in Sales
FF['CiS'] = df['sale'] - df['sale_lag1']
# % Change in Sales
FF['PCiS'] = (df['sale'] - df['sale_lag1']) / df['sale_lag1']
# Abnormal % Change in Sales
FF['APCiS'] = (((df['sale'] - df['sale_lag1']) / df['sale_lag1']) - (df['sum_sale'] - df['sum_sale_lag1']) / df['sum_sale_lag1'])
# Sales to Assets
FF['StA'] = (df['sale'] / df['at'])
# Change in Sales to Assets
FF['CiStA'] = (df['sale'] / df['at']) - (df['sale_lag1'] / df['at_lag1'])
# % Change in Sales to Assets
FF['PCiStA'] = ((df['sale'] / df['at']) - (df['sale_lag1'] / df['at_lag1']) / (df['sale_lag1'] / df['at_lag1']))
# Abnormal % Change in Sales to Assets
FF['APCiStA'] = ((((df['sale'] / df['at']) - (df['sale_lag1'] / df['at_lag1'])) / (df['sale_lag1'] / df['at_lag1'])) - ((df['sum_sale'] / df['sum_at']) - (df['sum_sale_lag1'] / df['sum_at_lag1'])) / (df['sum_sale_lag1'] / 
                                                                                                  df['sum_at_lag1']))
# Sales to Employees
FF['StE'] = df['sale'] / df['emp']
# Change in Sales to Employees
FF['CiSte'] = (df['sale'] / df['emp']) - (df['sale_lag1'] / df['emp_lag1'])
# % Change in Sales to Employees
FF['PCiStE'] = ((df['sale'] / df['emp']) - (df['sale_lag1'] / df['emp_lag1']) / (df['sale_lag1'] / df['emp_lag1']))
# Sales to Operating Expenses
FF['StOE'] = df['sale'] / df['xopr']
# Change in Sales to Operating Expenses
FF['CiStOE'] = (df['sale'] / df['xopr']) - (df['sale_lag1'] / df['xopr_lag1'])
# % Change in Sales to Operating Expenses
FF['PCiStOE'] = ((df['sale'] / df['xopr']) - (df['sale_lag1'] / df['xopr_lag1']) / (df['sale_lag1'] / df['xopr_lag1']))
# Abnormal % Change in Sales to Operating Expenses
FF['APCiStOE'] = ((((df['sale'] / df['xopr']) - (df['sale_lag1'] / df['xopr_lag1'])) / (df['sale_lag1'] / df['xopr_lag1'])) - (((df['sum_sale'] / df['sum_xopr']) - (df['sum_sale_lag1'] / df['sum_xopr_lag1'])) / (df['sum_sale_lag1'] /
                                                                                                       df['sum_xopr_lag1'])))
# Return on Assets
FF['ROA'] = df['ni'] / df['at']
# Change in Return on Assets
FF['CiROA'] = (df['ni'] / df['at']) - (df['ni_lag1'] / df['at_lag1'])
# % Change in Return on Assets
FF['PCiROA'] = ((df['ni'] / df['at']) - (df['ni_lag1'] / df['at_lag1']) / (df['ni_lag1'] / df['at_lag1']))
# Abnormal % Change in Return on Assets
FF['APCiROA'] = ((((df['ni'] / df['at']) - (df['ni_lag1'] / df['at_lag1'])) / (df['ni_lag1'] / df['at_lag1'])) - (((df['sum_ni'] / df['sum_at']) - (df['sum_ni_lag1'] / df['sum_at_lag1'])) / (df['sum_ni_lag1'] /
                                                                                                       df['sum_at_lag1'])))
# Return on Equity
FF['ROE'] = df['ni'] / df['ceq']
# Change in Return on Equity
FF['CiROE'] = (df['ni'] / df['ceq']) - (df['ni_lag1'] / df['ceq_lag1'])
# % Change in Return on Equity
FF['PCiROE'] = ((df['ni'] / df['ceq']) - (df['ni_lag1'] / df['ceq_lag1']) / (df['ni_lag1'] / df['ceq_lag1']))
# Abnormal % Change in Return on Equity
FF['APCiROE'] = ((((df['ni'] / df['ceq']) - (df['ni_lag1'] / df['ceq_lag1'])) / (df['ni_lag1'] / df['ceq_lag1'])) - (((df['sum_ni'] / df['sum_ceq']) - (df['sum_ni_lag1'] / df['sum_ceq_lag1'])) / (df['sum_ni_lag1'] /
                                                                                                       df['sum_ceq_lag1'])))
# Return on Sales
FF['ROS'] = df['ni'] / df['sale']
# Change in Return on Sales
FF['CiROS'] = (df['ni'] / df['sale']) - (df['ni_lag1'] / df['sale_lag1'])
# % Change in Return on Sales
FF['PCiROS'] = ((df['ni'] / df['sale']) - (df['ni_lag1'] / df['sale_lag1']) / (df['ni_lag1'] / df['sale_lag1']))
# Abnormal % Change in Return on Sales
FF['APCiROS'] = ((((df['ni'] / df['sale']) - (df['ni_lag1'] / df['sale_lag1'])) / (df['ni_lag1'] / df['sale_lag1'])) - (((df['sum_ni'] / df['sum_sale']) - (df['sum_ni_lag1'] / df['sum_sale_lag1'])) / (df['sum_ni_lag1'] /
                                                                                                       df['sum_sale_lag1'])))
# Accounts Payable to Inventory
FF['APtI'] = df['ap'] / df['invt']
# Change in Accounts Payable to Inventory
FF['CiAPtI'] = (df['ap'] / df['invt']) - (df['ap_lag1'] / df['invt_lag1'])
# % Change in Accounts Payable to Inventory
FF['PCiAPtI'] = ((df['ap'] / df['invt']) - (df['ap_lag1'] / df['invt_lag1']) / (df['ap_lag1'] / df['invt_lag1']))
# Abnormal % Change in Accounts Payable to Inventory
FF['APCiAPtI'] = ((((df['ap'] / df['invt']) - (df['ap_lag1'] / df['invt_lag1'])) / (df['ap_lag1'] / df['invt_lag1'])) - (((df['sum_ap'] / df['sum_invt']) - (df['sum_ap_lag1'] / df['sum_invt_lag1'])) / (df['sum_ap_lag1'] /
                                                                                                       df['sum_invt_lag1'])))
# Assets
FF['Assets'] = df['at']
# Change in Assets
FF['CiA'] = df['at'] - df['at_lag1']
# % Change in Assets
FF['PCiA'] = (df['at'] - df['at_lag1']) / df['at_lag1']
# Abnormal % Change in Assets
FF['APCiA'] = (((df['at'] - df['at_lag1']) / df['at_lag1']) - (df['sum_at'] - df['sum_at_lag1']) / df['sum_at_lag1'])
# Assets to Liabilities
FF['AtL'] = df['at'] / df['lt']
# Change in Assets to Liabilities
FF['CiAtL'] = (df['at'] / df['lt']) - (df['at_lag1'] / df['lt_lag1'])
# % Change in Assets to Liabilities
FF['PCiAtL'] = ((df['at'] / df['lt']) - (df['at_lag1'] / df['lt_lag1']) / (df['at_lag1'] / df['lt_lag1']))
# Abnormal % Change in Assets to Liabilities
FF['APCiAtL'] = ((((df['at'] / df['lt']) - (df['at_lag1'] / df['lt_lag1'])) / (df['at_lag1'] / df['lt_lag1'])) - (((df['sum_at'] / df['sum_lt']) - (df['sum_at_lag1'] / df['sum_lt_lag1'])) / (df['sum_at_lag1'] /
                                                                                                       df['sum_lt_lag1'])))
# Expenses
FF['Exp'] = df['xopr']
# Change in Expenses
FF['CiE'] = df['xopr'] - df['xopr_lag1']
# % Change in Expenses
FF['PCiE'] = (df['xopr'] - df['xopr_lag1']) / df['xopr_lag1']
# Abnormal % Change in Expenses
FF['APCiE'] = (((df['xopr'] - df['xopr_lag1']) / df['xopr_lag1']) - (df['sum_xopr'] - df['sum_xopr_lag1']) / df['sum_xopr_lag1'])
# Liabilities
FF['Liab'] = df['lt']

In [160]:
# Change in Liabilities
FF['CiL'] = df['lt'] - df['lt_lag1']
# % Change in Liabilities
FF['PCiL'] = (df['lt'] - df['lt_lag1']) / df['lt_lag1']
# Abnormal % Change in Liabilities
FF['APCiL'] = (((df['lt'] - df['lt_lag1']) / df['lt_lag1']) - (df['sum_lt'] - df['sum_lt_lag1']) / df['sum_lt_lag1'])
# Liabilities to Interest Expenses
FF['LtIE'] = df['lt'] / df['xint']
# Change in Liabilities to Interest Expenses
FF['CiLtIE'] = (df['lt'] / df['xint']) - (df['lt_lag1'] / df['xint_lag1'])
# % Change in Liabilities to Interest Expenses
FF['PCiLtIE'] = ((df['lt'] / df['xint']) - (df['lt_lag1'] / df['xint_lag1']) / (df['lt_lag1'] / df['xint_lag1']))
# Abnormal % Change in Liabilities to Interest Expenses
FF['APCiLtIE'] = ((((df['lt'] / df['xint']) - (df['lt_lag1'] / df['xint_lag1'])) / (df['lt_lag1'] / df['xint_lag1'])) - (((df['sum_lt'] / df['sum_xint']) - (df['sum_lt_lag1'] / df['sum_xint_lag1'])) / (df['sum_lt_lag1'] /
                                                                                                       df['sum_xint_lag1'])))

## Examing the Features

In [161]:
# Printing the dataset
FF

,cik,year,SIC,ARtS,ARtTA,AFDA,AFDAtAR,AFDAtNS,Zscore,Big4,...,PCiE,APCiE,Liab,CiL,PCiL,APCiL,LtIE,CiLtIE,PCiLtIE,APCiLtIE
0,1750,2003.0,5080,0.160533,0.147557,6.310,0.060290,0.009679,2.237914,1,...,NaN,NaN,407.608,NaN,NaN,NaN,21.659387,NaN,NaN,NaN
1,1750,2004.0,5080,0.169982,0.173608,5.863,0.046121,0.007840,2.730799,1,...,0.135007,-4.375779,417.486,9.878,0.024234,-1.772162,25.430103,3.770716,24.430103,-0.011345
4,1750,2005.0,5080,0.151872,0.139221,6.466,0.047449,0.007206,2.880039,1,...,0.173936,0.075357,556.102,138.616,0.332026,0.112553,30.887692,5.457589,29.887692,0.071530
7,1750,2006.0,5080,0.171218,0.170181,3.885,0.021382,0.003661,3.355914,1,...,0.169920,0.311159,573.390,17.288,0.031088,0.256937,32.435230,1.547539,31.435230,-0.196537
10,1750,2007.0,5080,0.146198,0.148657,5.977,0.029520,0.004316,2.752482,1,...,0.290933,0.183145,776.755,203.365,0.354671,0.171251,35.387472,2.952241,34.387472,0.106896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9264,1915657,2009.0,2911,0.165594,0.254464,NaN,NaN,NaN,2.617373,1,...,-0.169239,0.120475,1938.158,1000.265,1.066502,0.928741,44.508290,6.924920,43.508290,0.128355
9285,1915657,2010.0,2911,0.125455,0.282090,2.100,0.002011,0.000252,3.552323,1,...,0.706133,0.382693,2413.336,475.178,0.245170,0.091385,29.649319,-14.858971,28.649319,-0.410280
9309,1915657,2011.0,2911,0.099389,0.148771,3.500,0.002281,0.000227,3.207572,1,...,0.701848,0.421793,4478.721,2065.385,0.855822,0.751890,49.476056,19.826736,48.476056,0.748608
9332,1915657,2012.0,2911,0.035299,0.068660,2.500,0.003525,0.000124,5.141839,1,...,0.255013,0.218815,3686.339,-792.382,-0.176921,-0.215623,32.540111,-16.935945,31.540111,-0.289827


In [114]:
FF.describe()

,cik,year,SIC,ARtS,ARtTA,AFDA,AFDAtAR,AFDAtNS,Zscore,Big4,...,PCiE,APCiE,Liab,CiL,PCiL,APCiL,LtIE,CiLtIE,PCiLtIE,APCiLtIE
count,3.317900e+04,33179.000000,33179.000000,3.174200e+04,31994.000000,24595.000000,2.360800e+04,2.399400e+04,2.746500e+04,33179.000000,...,2.781900e+04,2.781900e+04,32641.000000,28913.000000,2.891100e+04,2.891100e+04,3.034400e+04,2.467700e+04,2.431900e+04,2.431800e+04
mean,9.315030e+05,2008.365442,4545.445884,inf,0.135021,17.883162,inf,inf,inf,0.775460,...,inf,inf,2597.651411,159.064931,inf,inf,inf,NaN,inf,inf
std,4.336661e+05,2.828214,2035.206072,NaN,0.112959,134.902378,NaN,NaN,NaN,0.417285,...,NaN,NaN,9854.483133,2265.180039,NaN,NaN,NaN,NaN,NaN,NaN
min,1.750000e+03,2003.000000,1000.000000,-2.286370e+00,0.000000,-0.484000,-7.464873e-03,-1.265816e-03,-1.067789e+04,0.000000,...,-1.000000e+00,-5.328990e+03,0.000000,-269999.000000,-9.981510e-01,-2.085744e+03,-1.700433e+03,-inf,-1.701433e+03,-1.401792e+02
25%,8.043280e+05,2006.000000,2911.000000,9.227037e-02,0.050357,0.216000,9.467169e-03,1.067403e-03,1.387595e+00,1.000000,...,-1.552170e-02,-6.956824e-02,44.088000,-9.238000,-6.405604e-02,-1.411990e-01,2.488719e+01,-7.389723e+00,2.280456e+01,-1.981000e-01
50%,1.018003e+06,2008.000000,3841.000000,1.463162e-01,0.111160,1.299000,2.211778e-02,3.063168e-03,2.901081e+00,1.000000,...,8.120038e-02,3.000028e-02,262.686000,3.831000,4.338786e-02,0.000000e+00,4.325883e+01,4.389777e-01,3.839643e+01,0.000000e+00
75%,1.224133e+06,2011.000000,6163.000000,2.028368e-01,0.188173,6.000000,4.680522e-02,6.671134e-03,5.006540e+00,1.000000,...,2.138416e-01,1.761161e-01,1314.588000,67.942000,2.449408e-01,1.984170e-01,1.258292e+02,9.879715e+00,8.242904e+01,2.740319e-01
max,1.915657e+06,2013.000000,9997.000000,inf,0.887772,5488.000000,inf,inf,inf,1.000000,...,inf,inf,460442.000000,69095.000000,inf,inf,inf,inf,inf,inf


## Keeping only one combination per CIK-Year

In [164]:
print('There are',FF[['cik','year']].duplicated().sum(),'duplicates of firm-year observations.')

There are 618 duplicates of firm-year observations.


In [165]:
print('Length of original dataset:',len(FF))
FF = FF.drop_duplicates(subset=['cik','year'])
print('Length of updated dataset',len(FF))

Length of original dataset: 33179
Length of updated dataset 32561


In [167]:
(FF['year'].astype(str) + '_' + FF['cik'].astype(str)).nunique()

32561

In [180]:
# Only considering the complete sample
year_cik_read = pd.read_csv('read_features.csv')
year_cik_read = year_cik_read[['cik','year']]
print(len(year_cik_read))
complete = pd.merge(year_cik_read, FF, on=['year','cik'], how='left')
print(len(complete))

5788
5788


In [181]:
print('There are',complete[['cik','year']].duplicated().sum(),'duplicates of firm-year observations.')

There are 5 duplicates of firm-year observations.


In [182]:
print('Length of original dataset:',len(complete))
complete = complete.drop_duplicates(subset=['cik','year'])
print('Length of updated dataset',len(complete))

Length of original dataset: 5788
Length of updated dataset 5783


In [183]:
(complete['year'].astype(str) + '_' + complete['cik'].astype(str)).nunique()

5783

## Taking care of Infinites

In [184]:
# Counting the amount of infintes per column
inf_cols = complete.columns[complete.isin([np.inf, -np.inf]).any()]
inf_count = complete[inf_cols].isin([np.inf, -np.inf]).sum()
inf_count

ARtS          19
cItS           6
DiRI          14
DtE            1
4GSGR         41
GM            10
cROE           1
ItS            7
TIE          431
UEP           19
PCiCM          1
PCiCS          9
PCiS          17
APCiS         17
APCiStA       17
StE           57
CiSte          3
APCiStOE      17
ROE            1
CiROE          2
PCiROE         1
APCiROE        1
ROS           38
CiROS         33
PCiROS        11
APCiROS       11
APtI        1472
CiAPtI        78
PCiAPtI       21
APCiAPtI      20
LtIE         431
CiLtIE       114
PCiLtIE       58
APCiLtIE      58
dtype: int64

The following columns contain infinite values:
1. Times Interest Earned (Interest and Related Expense - Total = 0)
2. Change in ROE
3. Accounts Payable to Inventory  (Inventory = 0)
4. Liabilities to Interest Expense (Interest and Related Expense - Total = 0)
5. Changes in APtI and LtIE 

As the CiROE has only 1 infinite value I decide to set this to 0.
As there are 1249 observations with no inventory, there is no accounts payable to inventory ratio, I decide to set these to 0.
As there are 530 observations with no xint, there is no TIE and no LtIE, I decide to set these to 0.

The changes are also set to 0 as they are likely influences by the infinity values of APtI and LtIE 

In [185]:
# Replacing infinites with 0's 
complete.replace([np.inf, -np.inf], 0, inplace=True)

# Counting the amount of infintes per column
inf_cols = complete.columns[complete.isin([np.inf, -np.inf]).any()]
inf_count = complete[inf_cols].isin([np.inf, -np.inf]).sum()
inf_count

Series([], dtype: float64)

## Taking care of missing values

In [186]:
# Counting the amount of missing values per column
na_cols_count = complete.isna().sum()
na_cols = complete.loc[:, na_cols_count > 0]
na_cols_count = na_cols.isna().sum()
print(na_cols_count)

SIC           22
ARtS          54
ARtTA         35
AFDA        1184
AFDAtAR     1247
            ... 
APCiL         40
LtIE         409
CiLtIE       839
PCiLtIE      895
APCiLtIE     895
Length: 103, dtype: int64


In [187]:
na_per_row = complete.isna().sum(axis=1)
most_na_per_row = na_per_row.idxmax()
complete.iloc[most_na_per_row].isna().sum()

103

There seems to be an observation with 103 missing values, I decide to exclude this observation from the final sample. Next there is an observation with 10 missing values. I decide to continue

In [188]:
print(len(complete))
complete = complete.drop([complete.index[most_na_per_row]])
print(len(complete))

5783
5782


In [189]:
# Counting the amount of missing values per column
na_cols_count = complete.isna().sum()
na_cols = complete.loc[:, na_cols_count > 0]
na_cols_count = na_cols.isna().sum()
print(na_cols_count)

SIC           21
ARtS          53
ARtTA         34
AFDA        1183
AFDAtAR     1246
            ... 
APCiL         39
LtIE         408
CiLtIE       838
PCiLtIE      894
APCiLtIE     894
Length: 103, dtype: int64


There seem to be 24 columns with missing values. Some of these columns have less than 1% of total observations as a missing value (1% = 68 observations). Most of them are changes or percentage changes. For these cases I do not consider the missing values to have a significant impact and decide to set them to 0.

In [190]:
nan_counts = complete.isna().sum()
cols_to_fill = nan_counts[nan_counts < 68].index.tolist()
complete[cols_to_fill] = complete[cols_to_fill].fillna(0)

In [191]:
# Counting the amount of missing values per column
na_cols_count = complete.isna().sum()
na_cols = complete.loc[:, na_cols_count > 0]
na_cols_count = na_cols.isna().sum()
print(na_cols_count)

AFDA        1183
AFDAtAR     1246
AFDAtNS     1211
Zscore       549
cItS         227
DiRI         193
4GSGR        230
HPRiVP        69
ItS           88
ROAtTA       150
TIE          401
TDA         1001
UEP          285
ACiOB       4657
RSST         589
CiFCF        688
CiI          118
CiOLA        817
CiR          152
DTaxE        174
PCiCM        223
PCiCS        174
WCA          357
CiS          150
PCiS         177
APCiS        177
CiStA        150
PCiStA       194
APCiStA      177
StE           75
CiSte        260
PCiStE       304
CiStOE       152
PCiStOE      194
APCiStOE     177
CiROA        150
PCiROA       150
APCiROA      150
CiROE        152
PCiROE       153
APCiROE      153
CiROS        172
PCiROS       194
APCiROS      194
CiAPtI      1517
PCiAPtI     1578
APCiAPtI    1579
CiE          150
PCiE         152
APCiE        152
LtIE         408
CiLtIE       838
PCiLtIE      894
APCiLtIE     894
dtype: int64


I decide to exclude the variable 'ACiOB' is it has missing values for more than 75% of the observations.

I decide to set the values for TDA RSST and WCA equal to the mean of the dataset as I believe setting these accrual related variables to 0 can negatively impact the data more than setting them to their mean.

I decide to approach the changes in APtI, FCF, and OLA and LtIE in the same way as I did with infinite values and decide to set these to 0. I also decide to set Deferred tax expense to 0 as it contains less than 2% of the observations.


In [192]:
complete = complete.drop(columns=['ACiOB'])

TDA_mean = complete['TDA'].mean()
RSST_mean = complete['RSST'].mean()
WCA_mean = complete['WCA'].mean()

complete['TDA'] = complete['TDA'].fillna(TDA_mean)
complete['RSST'] = complete['RSST'].fillna(RSST_mean)
complete['WCA'] = complete['WCA'].fillna(WCA_mean)

nan_counts = complete.isna().sum()
miss_cols = complete.loc[:, nan_counts > 0]
complete[miss_cols.columns] = complete[miss_cols.columns].fillna(0)

In [193]:
# Counting the amount of missing values per column
na_cols_count = complete.isna().sum()
na_cols = complete.loc[:, na_cols_count > 0]
na_cols_count = na_cols.isna().sum()
print(na_cols_count)

Series([], dtype: float64)


## Checking Types and Winsorizing

In [196]:
complete.dtypes[complete.dtypes != 'float']

year    int64
dtype: object

In [199]:
complete['cik'] = complete['cik'].astype('int')
complete['SIC'] = complete['SIC'].astype('int')
complete['ATO'] = complete['ATO'].astype('int')
complete['AI'] = complete['AI'].astype('int')
complete['Big4'] = complete['Big4'].astype('int')
complete['DCS'] = complete['DCS'].astype('int')
complete['DfF'] = complete['DfF'].astype('int')
complete['EoOL'] = complete['EoOL'].astype('int')
complete['PA'] = complete['PA'].astype('int')
complete['IoAR'] = complete['IoAR'].astype('int')
complete['IoGMP'] = complete['IoGMP'].astype('int')
complete['NewS'] = complete['NewS'].astype('int')
complete['dSIC'] = complete['dSIC'].astype('int')
complete['AMEX'] = complete['AMEX'].astype('int')

In [200]:
# Obtaining the numerical columns
num_cols = list(complete.select_dtypes(include=['float']).columns)

# Winsorizing variables
import numpy as np
from scipy.stats.mstats import winsorize

# 1% and 99%
limits = [0.01, 0.01]
winsorized = complete[num_cols].apply(lambda x: winsorize(x, limits=limits))

# Replacing the original with the winsorized
complete[num_cols] = winsorized

## Exporting the Features

In [201]:
# Counting the complete observations
complete_obs = complete.notna().all(axis=1).sum()
print('There are',complete_obs,'complete observations based on the financial fundamentals.')

There are 5782 complete observations based on the financial fundamentals.


In [202]:
# Exporting the data set
complete.to_csv('FF_features.csv', index=False)